<a href="https://colab.research.google.com/github/parthnaik13/NYRA/blob/main/1_1_NYRA_EDA_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>